# WGU Capstone: Stock Sentiment Analyzer Tool
                                -2022/12/04

### Main Codes 

In [ ]:
#import all the required packages 
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import yfinance as yf
from nrclex import NRCLex
from wordcloud import WordCloud, STOPWORDS #word clouds
from datetime import date
import mplfinance as mpf
import matplotlib.gridspec as gridspec

#main brain (processes the givin tickers )
def get_info(tickerHolder):    
    #scrape data from finviz
    finviz_url = 'https://finviz.com/quote.ashx?t='

    news_tables = {}
    url = finviz_url + tickerHolder
    req = Request(url=url, headers={'user-agent': 'my-app'})
    response = urlopen(req)
    html = BeautifulSoup(response, features='html.parser')
    news_table = html.find(id='news-table')
    news_tables[tickerHolder] = news_table

    #turn data into clean table
    parsed_data = []
    for ticker, news_table in news_tables.items():
        for row in news_table.findAll('tr'):
            title_tag = row.find('a')
            if title_tag:
                title = title_tag.text

            date_data = row.td.text.split() #save all date datas
            if len(date_data) == 1 :
                time = date_data[0]
            else:
                date = date_data[0]
                time = date_data[1]
            parsed_data.append([ticker, date, time, title])
    df = pd.DataFrame(parsed_data, columns=['ticker', 'date', 'time', 'title'])
    df = df[df['date']=='Today'] #filter for current 'Today' data
    df['time'] = pd.to_datetime(df.time).dt.time ## convert to corrct time format
    from datetime import date, timedelta
    date_when = date.today()# to save current date to save charts
    
    ##display results
    print(ticker)
    print(df[['time','title']]) #prints tickers dataframe

    ##ticker price chart
    security = yf.Ticker(tickerHolder)
    daily = security.history(period='ytd',interval='1d')
    daily = daily[['Open', 'High', 'Low', 'Close', 'Volume']]

    highs = daily['High'][:-1].max()
    lows = daily['Low'][:-1].min()
    
    ##vader sentiment analysis
    vader = SentimentIntensityAnalyzer()
    df['compound'] = df['title'].apply(lambda title: vader.polarity_scores(title)['compound'])
    mean_df = df.groupby(['ticker', 'time']).mean().unstack()
    
    senti_df = mean_df.xs('compound', axis="columns").transpose()

    ## emotion analysis
    df['emotions'] = df['title'].apply(lambda x: NRCLex(x).affect_frequencies) # get all the emotions
    df = pd.concat([df.drop(['emotions'], axis = 1), df['emotions'].apply(pd.Series)], axis = 1)  #droped emotions and get all 10 emotions
    emo_df = df.drop(['compound','anticip'],axis=1)
    emo_df = emo_df.groupby(['ticker']).sum()
    emo_df.fillna(0,inplace=True)

    ## word clouds
    topWords = 40
    stopwords = set(STOPWORDS)
    info = df['title'].str.cat(sep=' ')
    word_cloud = WordCloud(stopwords=stopwords,background_color="white",colormap='tab20b', max_words=topWords,width=600,height=400).generate(info)
    wc = word_cloud.to_image()

    ###fin ratio 
    fin_info = { 'M.Cap':security.fast_info['market_cap'],'Total Sh':security.fast_info['shares'],'Volume':security.fast_info['lastVolume'],'Avg. Vol':security.fast_info['threeMonthAverageVolume'],
            'YTD %':security.fast_info['yearChange'],'Chg %':100*(security.fast_info['lastPrice']/security.fast_info['regularMarketPreviousClose']-1),'Relative VOL':security.fast_info['lastVolume']/security.fast_info['threeMonthAverageVolume']}
    fin_sum = pd.DataFrame(fin_info.items(),columns=['key', 'value'])
    def human_format(num):
        magnitude = 0
        while abs(num) >= 1000:
            magnitude += 1
            num /= 1000.0
        # add more suffixes if you need them
        return '%.2f%s' % (num, ['', 'K', 'M', 'B', 'T', 'P'][magnitude])
    fin_sum['value'] = fin_sum['value'].apply(lambda x: human_format(x))

    #### plot charts
    
    bar_colors = ['black', 'tab:orange','tab:green','tab:blue','orange','tab:red','gray', 'brown','green','pink']
    
    fig = plt.figure(figsize=(12,12),constrained_layout=True)
    spec = gridspec.GridSpec(ncols=3, nrows=3, figure=fig)
    fig.suptitle(f"'{ticker}' Stock Analysis {date_when}", color='Black',fontsize=16,fontweight="bold")
    
    ax1 = fig.add_subplot(spec[0, :])
    ax2 = fig.add_subplot(spec[1,-2])
    ax6 = fig.add_subplot(spec[1,0])
    ax3 = fig.add_subplot(spec[1, -1])
    ax4 = fig.add_subplot(spec[2, 0])
    ax5 = fig.add_subplot(spec[2, 1:])
    
    #1
    mpf.plot(daily, ax=ax1, type='candle',style='charles', mav=(10,20,50),hlines=dict(hlines=[highs,(highs+lows)/2,lows],colors=['g','gray','r'],linestyle='-.')) #vlines='2023-01-01' vertical lines
    #mpf.plot(msft, ax=ax1, type='candle',style='charles', mav=(10,20,50),hlines=dict(hlines=[highs,(highs+lows)/2,lows],colors=['g','gray','r'],linestyle='-.')) #vlines='2023-01-01' vertical lines

    ax1.set_title('Daily - YTD Period ', loc='left', y=0.85, x=0.2)
    ax1.set_ylabel('')
    ax1.set_xticklabels('')
    #2
    finstats = security.earnings_dates.dropna().round(2) #[:10]
    finstats.rename({'EPS Estimate': 'Est.', 'Reported EPS':'Actual', 'Surprise(%)':'Sup(%)'}, axis=1, inplace=True)
    finstats.index.names = ['Date']
    finstats.index = finstats.index.strftime('%y/%m/%d')
    finstats.reset_index(inplace=True)

    ax2.table(cellText=finstats.values,colLabels=finstats.columns,loc='center')
    ax2.axis('off')
    ax2.set_title('Earnings',y=0.80, x=0.5)
    #fin sum
    ax6.table(cellText=fin_sum.values,loc='center')
    ax6.set_title('Financial Key Ratios',y=0.80, x=0.5)
    ax6.axis('off')
    #3
    ax3.imshow(wc)
    ax3.set_title('News: Top Mentioned',y=1)
    ax3.axis('off')
    
    #ticker.plot(ax=ax2, kind='line',title='Stats',color='Black')
    emo_df.T.plot(ax=ax4, kind='barh',title="Overall News Emotional Analysis",color=bar_colors,legend=False)
    senti_df.plot(ax=ax5, kind='bar',title='News Sentiment Analysis',legend=False,color='Black')
    #ax=axes[1,1].set_xlabel('')
    ax5.axhline(0, color = 'red')
    ax5.set_xlabel(' ')
    #ax5.set_xticklabels('')
    
    import os
    os.makedirs("Results", exist_ok = True) #Creates 'Results' folder in a same StockSentimentAnalyzerCapstone folder.
    return plt.savefig(f'Results/{tickerHolder}-{date_when}.png',dpi =300)
    #plt.savefig('results1.png',dpi=300)

### get tickers
tickers = [] #stores entered tickers in a tickers list.
while True:
    val = str(input("Enter a Ticker! or Just Enter to end the list.   "))
    if len(val) <= 0:
        break
    tickers.append(val.upper())
#ticker = val.upper()

#Runs all the enters in the tickers array.
for ticker in tickers:
    try:
        get_info(ticker)
    except Exception as e:
        print(f"{ticker} - Ticker is Invalid !!! (Note: ETFs and OTC stocks will not work.)")

#end